In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math
import pywt
import itertools
from scipy import stats

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


In [2]:
ipaDifficult = list()
ipaMedium = list()
ipaEasy = list()

In [3]:
TimeDwellOrig = 800
TimeFixation = 300

In [4]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [5]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [6]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [7]:
def FindTrialEndTimes(KeysSelected, timeTyping):
    # function to find start and end of tasks in experiments
    timeStartEnd = list() # format of this list will be: [startTime1, endTime1/startTime2, endTime2/startTime3, ..., endTimeN]
    
    timeStartEnd.append(timeTyping['startTime'])
    
    nTrial = 1
    
    for keys in KeysSelected:
        
        if keys[1] == 'NextPhrase':
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            timeStartEnd.append(endTimeTrial)
    
    
    timeStartEnd.append(timeTyping['endTime'])
    
    
    return timeStartEnd

In [8]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [9]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [10]:
def CreateTimeEpochsOfTrials(TimeStartEndMixed, UserKeys):
    # function to use list of mixed start and end times of trials and keys looked at by user to create trial epochs
    
    TimeEpochTrial = dict()
    TimeEpochTrial['Start'] = list()
    TimeEpochTrial['End'] = list()
    
    # Create list of times in userKeys to be able to use function 'nearestTimePoint'
    UserKeysStrTimes = [item3[0] for item3 in UserKeys]
    UserKeysTimes = timeConversion(UserKeysStrTimes)
    
    Flag_FoundSleepKey = 0 # Flag to indicate finding sleep key
    
    n = -1
    for time in TimeStartEndMixed:
        n = n + 1
        Flag_FoundSleepKey = 0
        
        if n == 0: # first time is only start time for the first trial
            TimeEpochTrial['Start'].append(time)
            continue
        elif n == len(TimeStartEndMixed)-1: # last time is only the end time for last trial
            
            TimeEpochTrial['End'].append(time)
            
        else: # the middle elements need to be divided into start and end
            TimeEpochTrial['End'].append(time)
            
            timeCheck = time
            
            # find the time in userkeys. Keep going to the previous element till you reach start of selection of
            # nextPhrase key
            while Flag_FoundSleepKey < 1:
                
                nearestToTrialStartTime, nearestToTrialStartInd = nearestTimePoint(UserKeysTimes, timeCheck)
                indCheck = nearestToTrialStartInd
                
                if 'NextPhrase' not in UserKeys[indCheck][1]:
                    TimeEpochTrial['Start'].append(nearestToTrialStartTime)
                    Flag_FoundSleepKey = 1
                    break
                else:
                    indCheck = indCheck - 2 # 2 added instead of 1, to allow nearestTimePoint to find the one before this
                    timeCheck = UserKeysTimes[indCheck]
                    
                
    return TimeEpochTrial      
            

In [11]:
def DwellTimeForBaseline(UserKeys_wDwellTime):
    
    DwellTime = list()
    
    for key in UserKeys_wDwellTime:
        if key[1] == 'NextPhrase':
            #print('NextPhrase found at ', key[2])
            if key[2] == 1:
                DwellTime.append(key[3])
                
    return DwellTime

In [12]:
def Convert2ColumnSizesTo1(GazeLog):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    PupilLogL = list()
    PupilLogR = list()
    
    PupilLogL_beforeDecimal = [item4[-5] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogL_afterDecimal = [item4[-4] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogR_beforeDecimal = [item4[-2] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogR_afterDecimal = [item4[-1] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    
    for i in range(0, len(PupilLogL_beforeDecimal)):
        if 'Valid' not in PupilLogL_beforeDecimal[i] and 'Valid' not in PupilLogL_afterDecimal[i]:
            if 'nan' not in PupilLogL_beforeDecimal[i] and 'nan' not in PupilLogL_afterDecimal[i]:
                PupilLogL.append(float(PupilLogL_beforeDecimal[i]+'.'+PupilLogL_afterDecimal[i]))
            else:
                PupilLogL.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            PupilLogL.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
    
    for i in range(0, len(PupilLogR_beforeDecimal)):
        if 'Valid' not in PupilLogR_beforeDecimal[i] and 'Valid' not in PupilLogR_afterDecimal[i]:
            if 'nan' not in PupilLogR_beforeDecimal[i] and 'nan' not in PupilLogR_afterDecimal[i]:
                PupilLogR.append(float(PupilLogR_beforeDecimal[i]+'.'+PupilLogR_afterDecimal[i]))
            else:
                PupilLogR.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            PupilLogR.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
            
    # if one of the pupils are nan, the other one is converted too
    nPupil = -1
    for pupilL in PupilLogL:
        nPupil = nPupil + 1
        if np.isnan(pupilL):
            if nPupil < len(PupilLogR):
                if not np.isnan(PupilLogR[nPupil]):
                    PupilLogR[nPupil] = np.nan
            else:
                del PupilLogL[len(PupilLogR):]
                
    nPupil = -1
    for pupilR in PupilLogR:
        nPupil = nPupil + 1
        if np.isnan(pupilR):
            if nPupil < len(PupilLogL):
                if not np.isnan(PupilLogL[nPupil]):
                    PupilLogL[nPupil] = np.nan
            else:
                del PupilLogR[len(PupilLogL):]
                
    #print(len(PupilLogL), len(PupilLogR))
    
    return PupilLogL, PupilLogR

In [13]:
def PupilSizeFromTrialTimes(TimeTrial, TimeGazeLog, TimeInternalGazeLog, PupilSizeLogL, PupilSizeLogR):
    # find pupil sizes from the start and end time given
    
    # find start and end time in gazeLog
    timeStart, timeStartInd = nearestTimePoint(TimeGazeLog, TimeTrial[0])
    timeEnd, timeEndInd = nearestTimePoint(TimeGazeLog, TimeTrial[1])
    
    pupilSize_TrialL = PupilSizeLogL[timeStartInd: timeEndInd]
    pupilSize_TrialR = PupilSizeLogR[timeStartInd: timeEndInd]
    
    TimeInternal_Trial = TimeInternalGazeLog[timeStartInd: timeEndInd]
    
    TimeGaze_Trial = TimeGazeLog[timeStartInd: timeEndInd]
    
    return pupilSize_TrialL, pupilSize_TrialR, TimeGaze_Trial, TimeInternal_Trial
    

In [14]:
def filterBlinks(pupilData, timeInDatetime_trial, timeInS_Trial):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    # recording extra blink information - duration and frequency
    blinkDurationList = list()
    blinkTimeList = list()
    blinkCount = 0
    nonBlinkCount = 0
    nonBlinkTimeList = list()
    timeRemove = 0
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (23 samples) before and after the blink will also be removed
    extraBlinkSamples = 23    
    
    # remove single missing data, that are due to hardware error
    missingVal_Single = np.argwhere(np.isnan(pupilData))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list
    missingVal_SingleDifference = [t - s for s, t in zip(missingVal_Single, missingVal_Single[1:])] # find difference 
    # between consecutive elements
    missingVal_SingleDifference.insert(0, missingVal_Single[0]) # insert the first blink index in the beginning of list
    
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index to 
    # the next nan value
    
    # first remove the single nan values, which are missing data
    eyeTracker_missingData = list() # list with index of single missing data  
    valInd = -1

    for val in missingVal_SingleDifference:
        valInd = valInd + 1
        if valInd == 0:
            continue
        if val != 1:
            if missingVal_SingleDifference[valInd-1] !=1: # if there are 2 consecutive missing values (denoted by 2 consecutive
                # non 1 numbers, they are added to the list of eyeTracker_missingData)
                eyeTracker_missingData.append(sum(missingVal_SingleDifference[:valInd]))
                
    # remove single missing values from pupil data
    pupilData_woSingleMissingData0 = [pupilData[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(pupilData))]
    pupilData_woSingleMissingData = [x for x in pupilData_woSingleMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woSingleMissingData0 = [timeInDatetime_trial[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(timeInDatetime_trial))]
    timeList_woSingleMissingData = [x for x in timeList_woSingleMissingData0 if x]
    
#     print(len(timeList_woSingleMissingData))
    
    
    timeInS_woSingleMissingData = timeInS_Trial[-1]-(len(timeList_woSingleMissingData)-len(timeInDatetime_trial))/90
    #print(timeInS_woSingleMissingData, timeInS_Trial[-1])
    
    # find the nan values again from pupilData_woSingleMissingData
    missingVal_Rest = np.argwhere(np.isnan(pupilData_woSingleMissingData))
    missingVal_Rest = list(itertools.chain.from_iterable(missingVal_Rest))
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    #print(missingVal_RestDifference)
    
    # compile and create list of start and end of blinks
    blink_missingData = dict()
    blink_missingData['Start'] = list()
    blink_missingData['End'] = list()
    
    valInd = -1
    for val in missingVal_RestDifference:
        valInd = valInd + 1
        if val > 1:
            
            
            #print('value', val)
            # instead of appending the actual index of blink start, since 250ms before and after the blink need to be
            # removed, it is also appended here.
            
            # just make sure that the additional samples do not make the index of blink go in negative
            if sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples > 0:
                
                blink_missingData['Start'].append(sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples)
            else:
                blink_missingData['Start'].append(0)
            
            if valInd == 0:
                lastBlinkStart = valInd
                continue
                
            # append blink duration list
            blinkDurationCurrent = valInd-lastBlinkStart
            # if blink duration is greater than 1s, it is not considered to be blink anymore
            if blinkDurationCurrent < 90: # since tobii sampling frequency is 90Hz
                blinkCount = blinkCount + 1
                blinkDurationList.append(blinkDurationCurrent/90)
                blinkTimeList.append(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])])
                lastBlinkStart = valInd
            else:
                # collect the time of non-blinks, that will need to be removed from trial time, to calculate 
                # blink frequency
                #print('current blink duration', valInd, lastBlinkStart, blinkDurationCurrent)
                timeRemove = timeRemove + blinkDurationCurrent
                nonBlinkCount = nonBlinkCount + 1
                nonBlinkTimeList.append(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])])
                lastBlinkStart = valInd
            
            # make sure that the additional samples do not increase the index to more than the length of the pupilData
            if sum(missingVal_RestDifference[:valInd])+extraBlinkSamples < len(pupilData_woSingleMissingData):
                blink_missingData['End'].append(sum(missingVal_RestDifference[:valInd])+extraBlinkSamples)
            else:
                blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
#         else:
#             # val is 1
#             if valInd-2 > 0 and valInd+3 < len(missingVal_RestDifference):
#                 if missingVal_RestDifference[valInd-1] > 1:
#                     if missingVal_RestDifference[valInd+1] == 1:
#                         if missingVal_RestDifference[valInd+2] > 1:
#                             print(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])], 
#                                 missingVal_RestDifference[valInd-2:valInd+3])
#                             if missingVal_RestDifference[valInd+2] > missingVal_RestDifference[valInd-1]:
#                                 if valInd-6>0:
#                                     print(missingVal_RestDifference[valInd-6:valInd+3])
#                     elif missingVal_RestDifference[valInd+1] > 1:
#                         print(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])], 
#                               missingVal_RestDifference[valInd-2:valInd+3])
#                         if missingVal_RestDifference[valInd+2] > missingVal_RestDifference[valInd-1]:
#                                 if valInd-6>0:
#                                     print(missingVal_RestDifference[valInd-6:valInd+3])
                        
                        
    # add the last blink index
    # make sure that the additional samples do not increase the index to more than the length of the pupilData
    if sum(missingVal_RestDifference)+extraBlinkSamples < len(pupilData_woSingleMissingData):
        blink_missingData['End'].append(sum(missingVal_RestDifference)+extraBlinkSamples)
    else:
        blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
                
    # need to create a list containing indexes that are to be removed
    blinkIndexList = list()
    
#     print(len(blink_missingData['Start']), len(blink_missingData['End']))
    
    # remove blinks and additional data from pupil data to get filtered data
    for indInd in range(0, len(blink_missingData['Start'])):
        blinkIndexList.append(range(blink_missingData['Start'][indInd], blink_missingData['End'][indInd]+1))
    # flatten the list
    blinkIndexList = list(itertools.chain.from_iterable(blinkIndexList))
    
    pupilData_woRestMissingData0 = [pupilData_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(pupilData_woSingleMissingData))]
    pupilData_filter = [x for x in pupilData_woRestMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woRestMissingData0 = [timeList_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(timeList_woSingleMissingData))]
    time_filter = [x for x in timeList_woRestMissingData0 if x]
    
    timeInS_Trial_filter = timeInS_woSingleMissingData-timeRemove/90
    
    blinkFrequency = blinkCount/timeInS_Trial_filter
    #print('freq', blinkFrequency, timeInS_woSingleMissingData, timeRemove)
    #print('time difference', len(timeInDatetime_trial), len(time_filter))
    if np.nan in pupilData_filter:
        print('nan values in filtered data')
#         for i in enumerate(pupilData_filter):
#             print(i)
        
    #print(nonBlinkCount, blinkCount, nonBlinkTimeList)
    return pupilData_filter, time_filter, blink_missingData, blinkDurationList, blinkFrequency, blinkTimeList, timeInS_Trial_filter

In [15]:
def modmax(d):
    # modulus maxima detection
    
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(0, len(d)):
        m[i] = math.fabs(d[i])
    
    # if value is larger than both neighbours , and strictly
    # larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(0, len(d)):
        ll = m[i-1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    #print(len(t))
    return t

In [16]:
def plotPupilSize(pupilData, timeData, TrialNumber):
    
    dataLenEqualizer = min(min(len(pupilData['Left']), len(pupilData['Right'])), len(timeData))
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(timeData[0:dataLenEqualizer], pupilData['Left'][0:dataLenEqualizer], 'b')
    ax.plot(timeData[0:dataLenEqualizer], pupilData['Right'][0:dataLenEqualizer], 'r')
    
    ax.set_ylabel('Absolute pupil size [in mm]')

    ax.set_title(TrialNumber)


In [17]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    #vals.plot()
    return(vals)

In [18]:
def ipaFunc(d):
    # compute ipa value of pupil diameter
    IPA = list()
    #print(len(d.pupildata.values))
    # obtain 2-level DWT of pupil diameter signal d
    
    # get signal duration (in seconds)
    tt = ((d.timestamp.values[-1] - d.timestamp.values[0]).item())/1000000000

    
    
    try:
        (cA2,cD2,cD1) = pywt.wavedec(d.pupildata.values,'sym12','symmetric', 2)
    except ValueError:
        print('value error in wavedec')
        return
        
    # normalize by 1=2j , j = 2 for 2-level DWT
    cA2[:] = [x / math.sqrt(4.0) for x in cA2]
    cD1[:] = [x / math.sqrt(2.0) for x in cD1]
    cD2[:] = [x / math.sqrt(4.0) for x in cD2]
    
    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    #print(len(cD2m))
    
    # threshold using universal threshold l_univ = s*sqrt(2logn)
    # where s is the standard deviation of the noise
    luniv = np.std(cD2m) * math.sqrt(2.0*np.log2(len(cD2m)))
    cD2t = pywt.threshold(cD2m ,luniv,mode="hard")
        
    # compute IPA
    ctr = 0
    for i in range(0, len(cD2t)):
        if math.fabs(cD2t[i]) > 0: ctr += 1
        #IPA = float(ctr)/tt
        # maybe each pupil data has an IPA?
    IPA = (float(ctr)/tt)
    
    return IPA, cD2m, cD2t, cD2, cD1, cA2, tt

In [19]:
def FindAndPlotPupilSizeForEpoch(GazeLog, DwellTimes_ForBaseline, TimeEpochTrial, ScoresDifficulty):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupilLogL, pupilLogR = Convert2ColumnSizesTo1(GazeLog)
    
    ipaList = list()
    timeOfGaze_TrialList = list()
    
    
    blinkDurationList = list()
    blinkDurationAverageList = list()
    blinkTimeList = list()
    blinkFrequencyList = list()
    timeInS_List = list()
    pupilMean_Absolute = list()
    pupilMean_Relative = list()
    interBlinkDurationList = list()
    
    # for every epoch, plot the pupil size
    for trialNr in range(0, len(timeEpochTrial['Start'])):
        if trialNr == 0:
            continue
        # find pupil sizes for the trial
        pupilSizeL_Trial, pupilSizeR_Trial, timeGaze_Trial, timeInternal_Trial = PupilSizeFromTrialTimes(
            [TimeEpochTrial['Start'][trialNr], TimeEpochTrial['End'][trialNr]], timeGazeLog, 
                                timeInternalGazeLog, pupilLogL, pupilLogR)
        
        pupilSize_Trial = dict()
        pupilSize_Filter = dict()
        pupilSize_woBlink = dict()
        
        # find difference in consecutive elements of internal time
        timeInternalDifference = [t - s for s, t in zip(timeInternal_Trial, timeInternal_Trial[1:])]
        # divide by 1000 to make it s
        timeOfGaze_Trial = [sum(timeInternalDifference[:i])/1000000 for i in range(1,len(timeInternalDifference))]

        # some trials were skipped, because the sentence was written before. If the time of trial is less than
        # 10s, the trial is skipped
        if timeOfGaze_Trial[-1] < 20:
            print('trial number ', trialNr+1, 'with', timeOfGaze_Trial[-1], 's will be skipped')
            continue
        
        pupilSize_Trial['Left'] = pupilSizeL_Trial
        pupilSize_Trial['Right'] = pupilSizeR_Trial
        
        #if trialNr == 4:
        #    for i in range(0, len(pupilSizeL_Trial)):
        #        print(pupilSizeL_Trial[i], pupilSizeR_Trial[i])
            
        #print('Trial', len(pupilSizeL_Trial), len(pupilSizeR_Trial))
        
        # filter the blinks
        pupilSizeL_woBlink, time_filter, missingPupilData, blinkDuration, blinkFrequency, blinkTimeList, timeInS_filter = filterBlinks(pupilSizeL_Trial, timeGaze_Trial, timeOfGaze_Trial)
        pupilSizeR_woBlink, time_filter, missingPupilData, blinkDuration, blinkFrequency, blinkTimeList, timeInS_filter = filterBlinks(pupilSizeR_Trial, timeGaze_Trial, timeOfGaze_Trial)
        
#         print(trialNr, blinkFrequency)
        #print(trialNr, blinkDuration)

        # time of trial
        timeInS_List.append(timeInS_filter)
        #print(trialNr, timeInS_filter)
        
        #print(index_blinkEndL)
        #print(index_blinkEndR)
        pupilSize_woBlink['Left'] = pupilSizeL_woBlink
        pupilSize_woBlink['Right'] = pupilSizeR_woBlink
        
        #print('After blink', len(pupilSizeL_woBlink), len(pupilSizeR_woBlink))
        # Hampel filter to remove the outliers
        winSize = 25
        pupilSizeL_filter = hampel(pupilSizeL_woBlink, winSize, 3)
        pupilSizeR_filter = hampel(pupilSizeR_woBlink, winSize, 3)

        pupilSize_Filter['Left'] = pupilSizeL_filter.values.tolist()
        pupilSize_Filter['Right'] = pupilSizeR_filter.values.tolist()
        
        pupilSizeL_filterList = [i[0] for i in pupilSizeL_filter.values]
        pupilSizeR_filterList = [i[0] for i in pupilSizeR_filter.values]
        
        #print('filter', len(pupilSizeL_filterList), len(pupilSizeR_filterList))
        RLCorrelation = np.corrcoef(pupilSizeL_filterList, pupilSizeR_filterList)
        if RLCorrelation[0][1] < 0.8:
            print('correlation between left and right:  ', RLCorrelation)
        
        # Relative Pupil Size Calculation 
        # First find baseline pupil size, which is the time when looking at NextPhrase key
        Samples_ForBaseline = int((int(DwellTimes_ForBaseline[trialNr-1][:-2])*90)/1000) # Number of samples of looking at key depend on
        
        #print(DwellTimes_ForBaseline[trialNr-1])
        
        # dwell time
        pupilL_baseline = np.mean(pupilSizeL_filterList[0:Samples_ForBaseline])
        pupilR_baseline = np.mean(pupilSizeR_filterList[0:Samples_ForBaseline])
        
        pupilL_Relative = [pupil/pupilL_baseline for pupil in pupilSizeL_filterList]
        pupilR_Relative = [pupil/pupilR_baseline for pupil in pupilSizeR_filterList]
        
        # average of whole trial
        #pupilL_avgRelative.append(np.mean(pupilL_Relative))
        #pupilR_avgRelative.append(np.mean(pupilR_Relative))
        
        # average of right and left
        pupilAvg_Relative = [((pupilL_Relative[i] + pupilR_Relative[i])/2) for i in range(0, min(len(pupilL_Relative), len(pupilR_Relative)))]
        
        pupilAvg_Absolute = [((pupilSizeL_filterList[i] + pupilSizeR_filterList[i])/2) for i in range(0, min(len(pupilSizeL_filterList), len(pupilSizeR_filterList)))]
        
        pupilLog_filter_wTime_Tuple = list(zip(time_filter, pupilAvg_Absolute))
        pupilAndTimeDf =  pd.DataFrame(pupilLog_filter_wTime_Tuple, columns=['timestamp','pupildata'])
        
        # compute IPA for the trial
        ipaVal, coeff_modmax, coeff_hard, coeff_D2, coeff_D1, coeff_A, timePeriodTrial = ipaFunc(pupilAndTimeDf)
        
        #print(trialNr+1, ipaVal, timeOfGaze_Trial[-1])
        
        ipaList.append(ipaVal)
        timeOfGaze_TrialList.append(timeOfGaze_Trial[-1])
        
        # find the inter blink duration
        interBlinkDuration = list()
        for i in range(1,len(blinkTimeList)):
            interBlinkDuration.append((blinkTimeList[i]-blinkTimeList[i-1]).total_seconds())
        if len(interBlinkDuration) > 1:
            #print(np.median(interBlinkDuration))
            interBlinkDurationList.append(np.median(interBlinkDuration))
        else:
            if interBlinkDuration:
                #print(interBlinkDuration)
                interBlinkDurationList.append(interBlinkDuration[0])
            else:
                interBlinkDurationList.append(0)
        
        blinkDurationList.append(blinkDuration)
        blinkFrequencyList.append(blinkFrequency)
        
        
        if len(blinkDuration)>0:
            blinkDurationAverageList.append(np.mean(blinkDuration))
        else:
            blinkDurationAverageList.append(0)
         
        pupilMean_Absolute.append(np.mean(pupilAvg_Absolute))
        pupilMean_Relative.append(np.mean(pupilAvg_Relative))
        
    return ipaList, pupilMean_Absolute, pupilMean_Relative, timeOfGaze_TrialList, blinkDurationList, blinkDurationAverageList, blinkFrequencyList, interBlinkDurationList, timeInS_List

In [20]:
blink_wChinRestFrequency = list()
blink_wChinRestDurationAverage = list()

time_wChinRestTrial = list()
blink_wChinRestDuration = list()
interBlinkInterval_wChinRest = list()

ipa_wChinRest = list()
pupilMeanAbsolute_wChinRest = list()
pupilMeanRelative_wChinRest = list()

score_wChinRestLIX = list()
score_wChinRestComplexity = list()
score_wChinRestDifficulty = list()
score_wChinRestSumOfScores = list()
score_wChinRestAfinn = list()

blink_woChinRestFrequency = list()
blink_woChinRestDurationAverage = list()

time_woChinRestTrial = list()
blink_woChinRestDuration = list()
interBlinkInterval_woChinRest = list()

ipa_woChinRest = list()
pupilMeanAbsolute_woChinRest = list()
pupilMeanRelative_woChinRest = list()

score_woChinRestLIX = list()
score_woChinRestComplexity = list()
score_woChinRestDifficulty = list()
score_woChinRestSumOfScores = list()
score_woChinRestAfinn = list()

subjName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['RLCorrelation'] = []

# extract self-reported scores list and LIX score of given sentence
file_name = r'C:/DTU/Data/201812_ExptToCheckMovementEffect/Data/Scores.xlsx'


#testNr = 'Test_wChinRest'
for root, dirs, subfolder in os.walk(subjName):
    scoreDifficult = list()
    scoreMedium = list()
    scoreEasy = list()
    # Semantic modeling score from afinn
    afinnDifficult = list()
    afinnMedium = list()
    afinnEasy = list()

    if not dirs:
        
        if 'tb' in root or 'trial' in root:
            continue
            
        userKeys = None
        gazeLog = None
        keysSelected = None
        
       
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_looks*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            
            elif fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
            
            elif fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])

                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or keysSelected is None or gazeLog is None:
                continue
            else:
                
                a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
                subjName = a.findall(root)[0]
                print('subject', subjName)
                
                sheet_to_df_map = pd.read_excel(file_name, sheet_name=subjName[0:2])
                
                testNr_re = re.compile('(?<=\\\Test)(.*)')
                testNr = 'Test' + testNr_re.findall(subjName)[0]
                if 'rh\Test_wChinRest' in subjName or 'sa\Test_woChinRest' in subjName:
                    testNr = 'Test' + testNr_re.findall(subjName)[0][:-3]
                #print(testNr)
                columnName1 = testNr + '_SumOfScores'
                columnName2 = testNr + '_LIX'
                columnName3 = testNr + '_Complexity'
                columnName4 = testNr + '_Difficulty'
                columnName5 = testNr + '_AfinnScore'
                
                
                # find _wChinRest of difficulty score
                scoresSumOfScores = sheet_to_df_map[columnName1]
                scoresSumOfScores = scoresSumOfScores[1:]

                # find _wChinRest of SumOfScores score
                scoresLIX = sheet_to_df_map[columnName2]
                scoresLIX = scoresLIX[1:]
                
                # find _wChinRest of SumOfScores score
                scoresComplexity = sheet_to_df_map[columnName3]
                scoresComplexity = scoresComplexity[1:]
                
                # find _wChinRest of SumOfScores score
                scoresDifficulty = sheet_to_df_map[columnName4]
                scoresDifficulty = scoresDifficulty[1:]
                
                # find affin score
                scoresAfinn = sheet_to_df_map[columnName5]
                scoresAfinn = scoresAfinn[1:]
                
                if subjName == 'sa\Test_woChinRest\p2':
                    userKeys = userKeys[:-1]
                    
                # fix userKeys due to comma related file changes
                userKeys_new = FixUserKeys(userKeys)
                
                # find dwell time of typing
                userKeys_wDwellTime = ComputeDwellTime(userKeys_new)
                
                # find start time of typing
                timeTyping = OptiKeyTypingTime(userKeys_new)
                
                # for some of the subjects, the data was not completely collected
                if subjName == 'sa\Test_woChinRest\p1' or subjName == 'rh\Test_wChinRest\p1':
                    del keysSelected[-1]
                
                # divide complete data into epochs of phrases
                timeStartEndMixed = FindTrialEndTimes(keysSelected, timeTyping)
                
                # create trial time epoch using the list of start/end times of trial and userKeys, to make sure that 
                # Sleep is completely there in every trial, to allow for baseline
                timeEpochTrial = CreateTimeEpochsOfTrials(timeStartEndMixed, userKeys_new)
                #print(timeEpochTrial)
                
                dwellTimes_ForBaseline = DwellTimeForBaseline(userKeys_wDwellTime)
                
                
                # find and plot pupil size for every trial
                ipaList, pupilMeanAbsolute, pupilMeanRelative, timeOfTrialList, blinkDuration, blinkDurationAverage, blinkFrequency, interBlinkInterval, time_trialList = FindAndPlotPupilSizeForEpoch(gazeLog, dwellTimes_ForBaseline, timeEpochTrial, scoresSumOfScores)
                
                if 'sa\Test_woChinRest' in subjName or 'rh\Test_wChinRest' in subjName:
                    if 'p1' in root:
                        ipaList1 = ipaList
                        pupilMeanAbsolute1 = pupilMeanAbsolute
                        pupilMeanRelative1 = pupilMeanRelative
                        timeOfTrialList1 = timeOfTrialList
                        blinkDurationAverage1 = blinkDurationAverage
                        blinkFrequency1 = blinkFrequency
                        time1 = time_trialList
                        blinkDuration1 = blinkDuration
                        interBlinkInterval1 = interBlinkInterval
                        continue
                        
                        #print('1', ipaList1)
                    else:
                        if 'sa\Test_woChinRest' in subjName:
                            ipaList2 = ipaList[1:]
                            pupilMeanAbsolute2 = pupilMeanAbsolute[1:]
                            pupilMeanRelative2 = pupilMeanRelative[1:]
                            timeOfTrialList2 = timeOfTrialList[1:]
                            blinkDurationAverage2 = blinkDurationAverage[1:]
                            blinkFrequency2 = blinkFrequency[1:]
                            time2 = time_trialList[1:]
                            blinkDuration2 = blinkDuration[1:]
                            interBlinkInterval2 = interBlinkInterval[1:]
                        else:
                            ipaList2 = ipaList
                            pupilMeanAbsolute2 = pupilMeanAbsolute
                            pupilMeanRelative2 = pupilMeanRelative
                            timeOfTrialList2 = timeOfTrialList
                            blinkDurationAverage2 = blinkDurationAverage
                            blinkFrequency2 = blinkFrequency
                            time2 = time_trialList
                            blinkDuration2 = blinkDuration
                            interBlinkInterval2 = interBlinkInterval
                            
                            
                    blinkFrequency = blinkFrequency1 + blinkFrequency2
                    blinkDurationAverage = blinkDurationAverage1 + blinkDurationAverage2
                    time_trialList = time1 + time2
                    blinkDuration = blinkDuration1 + blinkDuration2
                    interBlinkInterval = interBlinkInterval1 + interBlinkInterval2
                    ipaList = ipaList1 + ipaList2
                    timeOfTrialList = timeOfTrialList1 + timeOfTrialList2
                    pupilMeanAbsolute = pupilMeanAbsolute1 + pupilMeanAbsolute2
                    pupilMeanRelative = pupilMeanRelative1 + pupilMeanRelative2
                    
                timeOfTrialListPlot = [i/1000 for i in timeOfTrialList]


                if 'Test_wChinRest' in subjName:
                    blink_wChinRestFrequency.append(blinkFrequency)
                    blink_wChinRestDurationAverage.append(blinkDurationAverage)
                    time_wChinRestTrial.append(time_trialList)
                    blink_wChinRestDuration.append(blinkDuration)
                    interBlinkInterval_wChinRest.append(interBlinkInterval)
                    ipa_wChinRest.append(ipaList)
                    pupilMeanAbsolute_wChinRest.append(pupilMeanAbsolute)
                    pupilMeanRelative_wChinRest.append(pupilMeanRelative)
                    
                    score_wChinRestLIX.append(scoresLIX)
                    score_wChinRestComplexity.append(scoresComplexity)
                    score_wChinRestDifficulty.append(scoresDifficulty)
                    score_wChinRestSumOfScores.append(scoresSumOfScores)
                    score_wChinRestAfinn.append(scoresAfinn)
                else:
                    blink_woChinRestFrequency.append(blinkFrequency)
                    blink_woChinRestDurationAverage.append(blinkDurationAverage)
                    time_woChinRestTrial.append(time_trialList)
                    blink_woChinRestDuration.append(blinkDuration)
                    interBlinkInterval_woChinRest.append(interBlinkInterval)
                    ipa_woChinRest.append(ipaList)
                    pupilMeanAbsolute_woChinRest.append(pupilMeanAbsolute)
                    pupilMeanRelative_woChinRest.append(pupilMeanRelative)
                    
                    score_woChinRestLIX.append(scoresLIX)
                    score_woChinRestComplexity.append(scoresComplexity)
                    score_woChinRestDifficulty.append(scoresDifficulty)
                    score_woChinRestSumOfScores.append(scoresSumOfScores)
                    score_woChinRestAfinn.append(scoresAfinn)
                

subject bh\Test_wChinRest
trial number  11 with 4.596693 s will be skipped
subject bh\Test_woChinRest
trial number  11 with 7.339166 s will be skipped
subject ph\Test_wChinRest
trial number  11 with 18.00399 s will be skipped
subject ph\Test_woChinRest
correlation between left and right:   [[1.         0.78966669]
 [0.78966669 1.        ]]
subject pt\Test_wChinRest
subject pt\Test_woChinRest
trial number  11 with 17.47867 s will be skipped
subject rh\Test_wChinRest\p1
subject rh\Test_wChinRest\p2
trial number  4 with 10.770031 s will be skipped
trial number  10 with 14.989218 s will be skipped
subject rh\Test_woChinRest
correlation between left and right:   [[1.         0.79904233]
 [0.79904233 1.        ]]
trial number  11 with 10.476645 s will be skipped
subject sa\Test_wChinRest
subject sa\Test_woChinRest\p1
subject sa\Test_woChinRest\p2
trial number  3 with 8.449478 s will be skipped
trial number  5 with 8.926912 s will be skipped


In [24]:
# save the data into excel files by first saving them in pandas dataframe

df_wChinRest = pd.DataFrame({'blink_frequency_wChinRest': blink_wChinRestFrequency,
                             'blink_duration_average_wChinRest': blink_wChinRestDurationAverage,
                             'blink_duration_wChinRest': blink_wChinRestDuration,
                             #'inter_blink_duration_wChinRest':interBlinkInterval_wChinRest, 
                             'pupil_relative_wChinRest': pupilMeanRelative_wChinRest,
                             'pupil_absolute_wChinRest': pupilMeanAbsolute_wChinRest,
                             'ipa_wChinRest' : ipa_wChinRest,
                             'score_complexity_wChinRest': score_wChinRestComplexity,
                             'score_difficulty_wChinRest' : score_wChinRestDifficulty,
                             'score_LIX_wChinRest': score_wChinRestLIX})

df_woChinRest = pd.DataFrame({'blink_frequency_woChinRest': blink_woChinRestFrequency,
                             'blink_duration_average_woChinRest': blink_woChinRestDurationAverage,
                             #'blink_duration_woChinRest': blink_woChinRestDuration,
                             'inter_blink_duration_woChinRest':interBlinkInterval_woChinRest, 
                             'pupil_relative_woChinRest': pupilMeanRelative_woChinRest,
                             'pupil_absolute_woChinRest': pupilMeanAbsolute_woChinRest,
                             'ipa_woChinRest' : ipa_woChinRest,
                             'score_complexity_woChinRest': score_woChinRestComplexity,
                             'score_difficulty_woChinRest' : score_woChinRestDifficulty,
                             'score_LIX_woChinRest': score_woChinRestLIX})

In [25]:
participant_id = [1, 2, 3, 4, 5]
n_trials = 9

participant_id_list = [[1]*n_trials, [2]*n_trials, [3]*n_trials, [4]*n_trials, [5]*n_trials]


wChinRest = [1]*n_trials*len(participant_id)
participant_id_wChinRest = [item for sublist in participant_id_list for item in sublist]
blinkFrequency_wChinRest1 = [item for sublist in blink_wChinRestFrequency for item in sublist]
blinkDurationAverage_wChinRest1 = [item for sublist in blink_wChinRestDurationAverage for item in sublist]
interBlinkInterval_wChinRest1 = [item for sublist in interBlinkInterval_wChinRest for item in sublist]
pupilRelative_wChinRest1 = [item for sublist in pupilMeanRelative_wChinRest for item in sublist]
pupilAbsolute_wChinRest1 = [item for sublist in pupilMeanAbsolute_wChinRest for item in sublist]
ipa_wChinRest1 = [item for sublist in ipa_wChinRest for item in sublist]
scoreComplexity_wChinRest1 = [item for sublist in score_wChinRestComplexity for item in sublist]
scoreDifficulty_wChinRest1 = [item for sublist in score_wChinRestDifficulty for item in sublist]
scoreLIX_wChinRest1 = [item for sublist in score_wChinRestLIX for item in sublist]



woChinRest = [0]*n_trials*len(participant_id)
participant_id_woChinRest = [item for sublist in participant_id_list for item in sublist]
blinkFrequency_woChinRest1 = [item for sublist in blink_woChinRestFrequency for item in sublist]
blinkDurationAverage_woChinRest1 = [item for sublist in blink_woChinRestDurationAverage for item in sublist]
interBlinkInterval_woChinRest1 = [item for sublist in interBlinkInterval_woChinRest for item in sublist]
pupilRelative_woChinRest1 = [item for sublist in pupilMeanRelative_woChinRest for item in sublist]
pupilAbsolute_woChinRest1 = [item for sublist in pupilMeanAbsolute_woChinRest for item in sublist]
ipa_woChinRest1 = [item for sublist in ipa_woChinRest for item in sublist]
scoreComplexity_woChinRest1 = [item for sublist in score_woChinRestComplexity for item in sublist]
scoreDifficulty_woChinRest1 = [item for sublist in score_woChinRestDifficulty for item in sublist]
scoreLIX_woChinRest1 = [item for sublist in score_woChinRestLIX for item in sublist]




In [26]:
# concatenate lists 

participant_id_total = participant_id_wChinRest + participant_id_woChinRest
chinRest_total = wChinRest + woChinRest

blinkFrequency_total = blinkFrequency_wChinRest1 + blinkFrequency_woChinRest1
blinkDurationAverage_total = blinkDurationAverage_wChinRest1 + blinkDurationAverage_woChinRest1
interBlinkInterval_total = interBlinkInterval_wChinRest1 + interBlinkInterval_woChinRest1
pupilRelative_total = pupilRelative_wChinRest1 + pupilRelative_woChinRest1
pupilAbsolute_total = pupilAbsolute_wChinRest1 + pupilAbsolute_woChinRest1
ipa_total = ipa_wChinRest1 + ipa_woChinRest1

scoreComplexity_total = scoreComplexity_wChinRest1 + scoreComplexity_woChinRest1
scoreDifficulty_total = scoreDifficulty_wChinRest1 + scoreDifficulty_woChinRest1
scoreLIX_total = scoreLIX_wChinRest1 + scoreLIX_woChinRest1






In [27]:
# create pandas data frame

df_data = pd.DataFrame({'participant_id': participant_id_total,
                        'chin_rest': chinRest_total,
                        'blink_frequency': blinkFrequency_total,
                        'blink_duration_average': blinkDurationAverage_total,
                        'inter_blink_duration': interBlinkInterval_total,
                        'pupil_relative': pupilRelative_total,
                        'pupil_absolute': pupilAbsolute_total,
                        'ipa': ipa_total,
                        'score_complexity': scoreComplexity_total,
                        'score_difficulty': scoreDifficulty_total,
                        'score_LIX': scoreLIX_total                      
                       })


In [28]:
# save data to file, to process in R

df_data.to_excel('C:\\DTU\\Data\\201812_ExptToCheckMovementEffect\\Data\\python_data.xlsx', index=False)
#df_woChinRest.to_excel(writer, sheet_name='woChinRest', index=False)

#writer.save()
#writer.close()

In [22]:
scoreComplexity_wChinRest_difficult = list()
scoreDifficulty_wChinRest_difficult = list()
scoreAfinn_wChinRest_difficult = list()
            
blinkFrequency_wChinRest_difficult = list()
blinkDurationAverage_wChinRest_difficult = list()
interBlinkInterval_wChinRest_difficult = list()

ipa_wChinRest_difficult = list()
pupilMeanAbsolute_wChinRest_difficult = list()
pupilMeanRelative_wChinRest_difficult = list()
            
scoreComplexity_wChinRest_easy = list()
scoreDifficulty_wChinRest_easy = list()
scoreAfinn_wChinRest_easy = list()
            
blinkFrequency_wChinRest_easy = list()
blinkDurationAverage_wChinRest_easy = list()
interBlinkInterval_wChinRest_easy = list()

ipa_wChinRest_easy = list()
pupilMeanAbsolute_wChinRest_easy = list()
pupilMeanRelative_wChinRest_easy = list()


subjInd = -1
for subjScores in score_wChinRestLIX:
    subjInd = subjInd + 1
    
    scoreComplexity_subj_diffiult = list()
    scoreDifficulty_subj_difficult = list()
    scoreAfinn_subj_difficult = list()
    blinkFrequency_subj_difficult = list()
    blinkDurationAverage_subj_difficult = list()
    interBlinkInterval_subj_difficult= list()
    ipa_subj_difficult = list()
    pupilMeanAbsolute_subj_difficult = list()
    pupilMeanRelative_subj_difficult = list()
    
    scoreComplexity_subj_easy = list()
    scoreDifficulty_subj_easy = list()
    scoreAfinn_subj_easy = list()
    blinkFrequency_subj_easy = list()
    blinkDurationAverage_subj_easy = list()
    interBlinkInterval_subj_easy = list()
    ipa_subj_easy = list()
    pupilMeanAbsolute_subj_easy = list()
    pupilMeanRelative_subj_easy = list()
    
    scoreInd = -1
    blinkDurationList = list()
    for score in subjScores:
        scoreInd = scoreInd + 1
        if score > 2:
            
            scoreComplexity_subj_diffiult.append(score_wChinRestComplexity[subjInd][scoreInd+1])
            scoreDifficulty_subj_difficult.append(score_wChinRestDifficulty[subjInd][scoreInd+1])
            scoreAfinn_subj_difficult.append(score_wChinRestAfinn[subjInd][scoreInd+1])
            
            blinkFrequency_subj_difficult.append(blink_wChinRestFrequency[subjInd][scoreInd])
            blinkDurationAverage_subj_difficult.append(blink_wChinRestDurationAverage[subjInd][scoreInd])
            interBlinkInterval_subj_difficult.append(interBlinkInterval_wChinRest[subjInd][scoreInd])
            
            ipa_subj_difficult.append(ipa_wChinRest[subjInd][scoreInd])
            pupilMeanAbsolute_subj_difficult.append(pupilMeanAbsolute_wChinRest[subjInd][scoreInd])
            pupilMeanRelative_subj_difficult.append(pupilMeanRelative_wChinRest[subjInd][scoreInd])
            
        elif score < 2:
            scoreComplexity_subj_easy.append(score_wChinRestComplexity[subjInd][scoreInd+1])
            scoreDifficulty_subj_easy.append(score_wChinRestDifficulty[subjInd][scoreInd+1])
            scoreAfinn_subj_easy.append(score_wChinRestAfinn[subjInd][scoreInd+1])
            
            blinkFrequency_subj_easy.append(blink_wChinRestFrequency[subjInd][scoreInd])
            blinkDurationAverage_subj_easy.append(blink_wChinRestDurationAverage[subjInd][scoreInd])
            interBlinkInterval_subj_easy.append(interBlinkInterval_wChinRest[subjInd][scoreInd])
            
            ipa_subj_easy.append(ipa_wChinRest[subjInd][scoreInd])
            pupilMeanAbsolute_subj_easy.append(pupilMeanAbsolute_wChinRest[subjInd][scoreInd])
            pupilMeanRelative_subj_easy.append(pupilMeanRelative_wChinRest[subjInd][scoreInd])
        
        if scoreInd == len(subjScores)-1:
            scoreComplexity_wChinRest_difficult.append(np.mean(scoreComplexity_subj_diffiult))
            scoreDifficulty_wChinRest_difficult.append(np.mean(scoreDifficulty_subj_difficult))
            scoreAfinn_wChinRest_difficult.append(np.mean(scoreAfinn_subj_difficult))
            
            blinkFrequency_wChinRest_difficult.append(np.mean(blinkFrequency_subj_difficult))
            blinkDurationAverage_wChinRest_difficult.append(np.mean(blinkDurationAverage_subj_difficult))
            interBlinkInterval_wChinRest_difficult.append(np.mean(interBlinkInterval_subj_difficult))
            
            ipa_wChinRest_difficult.append(np.mean(ipa_subj_difficult))
            pupilMeanAbsolute_wChinRest_difficult.append(np.mean(pupilMeanAbsolute_subj_difficult))
            pupilMeanRelative_wChinRest_difficult.append(np.mean(pupilMeanRelative_subj_difficult))
            
            scoreComplexity_wChinRest_easy.append(np.mean(scoreComplexity_subj_easy))
            scoreDifficulty_wChinRest_easy.append(np.mean(scoreDifficulty_subj_easy))
            scoreAfinn_wChinRest_easy.append(np.mean(scoreAfinn_subj_easy))
            
            blinkFrequency_wChinRest_easy.append(np.mean(blinkFrequency_subj_easy))
            blinkDurationAverage_wChinRest_easy.append(np.mean(blinkDurationAverage_subj_easy))
            interBlinkInterval_wChinRest_easy.append(np.mean(interBlinkInterval_subj_easy))
            
            ipa_wChinRest_easy.append(np.mean(ipa_subj_easy))
            pupilMeanAbsolute_wChinRest_easy.append(np.mean(pupilMeanAbsolute_subj_easy))
            pupilMeanRelative_wChinRest_easy.append(np.mean(pupilMeanRelative_subj_easy))
            
            

In [23]:
print('scores')
print(stats.ttest_rel(scoreComplexity_wChinRest_difficult, scoreComplexity_wChinRest_easy))
print(stats.ttest_rel(scoreDifficulty_wChinRest_difficult, scoreDifficulty_wChinRest_easy))
print(stats.ttest_rel(scoreAfinn_wChinRest_difficult, scoreAfinn_wChinRest_easy))

print('blinks')
print(stats.ttest_rel(blinkFrequency_wChinRest_difficult, blinkFrequency_wChinRest_easy))
print(stats.ttest_rel(blinkDurationAverage_wChinRest_difficult, blinkDurationAverage_wChinRest_easy))
print(stats.ttest_rel(interBlinkInterval_wChinRest_difficult, interBlinkInterval_wChinRest_easy))

print('pupils')
print(stats.ttest_rel(ipa_wChinRest_difficult, ipa_wChinRest_easy))
print(stats.ttest_rel(pupilMeanAbsolute_wChinRest_difficult, pupilMeanAbsolute_wChinRest_easy))
print(np.mean(pupilMeanAbsolute_wChinRest_difficult), np.mean(pupilMeanAbsolute_wChinRest_easy))

print(stats.ttest_rel(pupilMeanRelative_wChinRest_difficult, pupilMeanRelative_wChinRest_easy))
print(np.mean(pupilMeanRelative_wChinRest_difficult), np.mean(pupilMeanRelative_wChinRest_easy))

scores
Ttest_relResult(statistic=1.49071198499986, pvalue=0.21029538311834686)
Ttest_relResult(statistic=2.502022303445038, pvalue=0.06662242870964402)
Ttest_relResult(statistic=-0.30151134457776363, pvalue=0.7780495489000209)
blinks
Ttest_relResult(statistic=0.6378192332197945, pvalue=0.5582742707497621)
Ttest_relResult(statistic=0.16404470253083198, pvalue=0.8776514091186837)
Ttest_relResult(statistic=-0.45598220838619774, pvalue=0.6720611898465632)
pupils
Ttest_relResult(statistic=-1.72224002503088, pvalue=0.16012971306289278)
Ttest_relResult(statistic=2.4185064398552663, pvalue=0.0728806874042725)
5.040391565699416 4.926015271295617
Ttest_relResult(statistic=2.036091051185982, pvalue=0.11143780892444397)
1.0295867735742905 1.0023643160739524


In [25]:
print('scores')
print(stats.ttest_rel(scoreComplexity_woChinRest_difficult, scoreComplexity_woChinRest_easy))
print(stats.ttest_rel(scoreDifficulty_woChinRest_difficult, scoreDifficulty_woChinRest_easy))
print(stats.ttest_rel(scoreAfinn_woChinRest_difficult, scoreAfinn_woChinRest_easy))

print('blinks')
print(stats.ttest_rel(blinkFrequency_woChinRest_difficult, blinkFrequency_woChinRest_easy))
print(stats.ttest_rel(blinkDurationAverage_woChinRest_difficult, blinkDurationAverage_woChinRest_easy))
print(stats.ttest_rel(interBlinkInterval_woChinRest_difficult, interBlinkInterval_woChinRest_easy))

print('pupils')
print(stats.ttest_rel(ipa_woChinRest_difficult, ipa_woChinRest_easy))
print(stats.ttest_rel(pupilMeanAbsolute_woChinRest_difficult, pupilMeanAbsolute_woChinRest_easy))
print(np.mean(pupilMeanAbsolute_woChinRest_difficult), np.mean(pupilMeanAbsolute_woChinRest_easy))

print(stats.ttest_rel(pupilMeanRelative_woChinRest_difficult, pupilMeanRelative_woChinRest_easy))
print(np.mean(pupilMeanRelative_woChinRest_difficult), np.mean(pupilMeanRelative_woChinRest_easy))

scores
Ttest_relResult(statistic=10.06230589874906, pvalue=0.0005486474801839636)
Ttest_relResult(statistic=9.130087106158284, pvalue=0.0007985330691842276)
Ttest_relResult(statistic=2.2499999999999996, pvalue=0.08764517650339469)
blinks
Ttest_relResult(statistic=2.8772898324005354, pvalue=0.045136030336156444)
Ttest_relResult(statistic=1.0745675745448307, pvalue=0.3430680792342676)
Ttest_relResult(statistic=-0.15567673989850295, pvalue=0.8838282289943773)
pupils
Ttest_relResult(statistic=-3.313182009310074, pvalue=0.029566315355539524)
Ttest_relResult(statistic=6.4085506789438265, pvalue=0.003045853466641179)
5.155802430328728 5.025327769930554
Ttest_relResult(statistic=-0.49555269783402534, pvalue=0.6462000977564855)
1.0432966768769085 1.0457177142063778


In [26]:
print(1.0457177142063778 - 1.0432966768769085)

0.0024210373294693


In [24]:
scoreComplexity_woChinRest_difficult = list()
scoreDifficulty_woChinRest_difficult = list()
scoreAfinn_woChinRest_difficult = list()
            
blinkFrequency_woChinRest_difficult = list()
blinkDurationAverage_woChinRest_difficult = list()
interBlinkInterval_woChinRest_difficult = list()

ipa_woChinRest_difficult = list()
pupilMeanAbsolute_woChinRest_difficult = list()
pupilMeanRelative_woChinRest_difficult = list()
            
scoreComplexity_woChinRest_easy = list()
scoreDifficulty_woChinRest_easy = list()
scoreAfinn_woChinRest_easy = list()
            
blinkFrequency_woChinRest_easy = list()
blinkDurationAverage_woChinRest_easy = list()
interBlinkInterval_woChinRest_easy = list()

ipa_woChinRest_easy = list()
pupilMeanAbsolute_woChinRest_easy = list()
pupilMeanRelative_woChinRest_easy = list()


subjInd = -1
for subjScores in score_woChinRestLIX:
    subjInd = subjInd + 1
    
    scoreComplexity_subj_diffiult = list()
    scoreDifficulty_subj_difficult = list()
    scoreAfinn_subj_difficult = list()
    blinkFrequency_subj_difficult = list()
    blinkDurationAverage_subj_difficult = list()
    interBlinkInterval_subj_difficult= list()
    ipa_subj_difficult = list()
    pupilMeanAbsolute_subj_difficult = list()
    pupilMeanRelative_subj_difficult = list()
    
    scoreComplexity_subj_easy = list()
    scoreDifficulty_subj_easy = list()
    scoreAfinn_subj_easy = list()
    blinkFrequency_subj_easy = list()
    blinkDurationAverage_subj_easy = list()
    interBlinkInterval_subj_easy = list()
    ipa_subj_easy = list()
    pupilMeanAbsolute_subj_easy = list()
    pupilMeanRelative_subj_easy = list()
    
    scoreInd = -1
    blinkDurationList = list()
    for score in subjScores:
        scoreInd = scoreInd + 1
        if score > 2:
            
            scoreComplexity_subj_diffiult.append(score_woChinRestComplexity[subjInd][scoreInd+1])
            scoreDifficulty_subj_difficult.append(score_woChinRestDifficulty[subjInd][scoreInd+1])
            scoreAfinn_subj_difficult.append(score_woChinRestAfinn[subjInd][scoreInd+1])
            
            blinkFrequency_subj_difficult.append(blink_woChinRestFrequency[subjInd][scoreInd])
            blinkDurationAverage_subj_difficult.append(blink_woChinRestDurationAverage[subjInd][scoreInd])
            interBlinkInterval_subj_difficult.append(interBlinkInterval_woChinRest[subjInd][scoreInd])
            
            ipa_subj_difficult.append(ipa_woChinRest[subjInd][scoreInd])
            pupilMeanAbsolute_subj_difficult.append(pupilMeanAbsolute_woChinRest[subjInd][scoreInd])
            pupilMeanRelative_subj_difficult.append(pupilMeanRelative_woChinRest[subjInd][scoreInd])
            
        elif score < 2:
            scoreComplexity_subj_easy.append(score_woChinRestComplexity[subjInd][scoreInd+1])
            scoreDifficulty_subj_easy.append(score_woChinRestDifficulty[subjInd][scoreInd+1])
            scoreAfinn_subj_easy.append(score_woChinRestAfinn[subjInd][scoreInd+1])
            
            blinkFrequency_subj_easy.append(blink_woChinRestFrequency[subjInd][scoreInd])
            blinkDurationAverage_subj_easy.append(blink_woChinRestDurationAverage[subjInd][scoreInd])
            interBlinkInterval_subj_easy.append(interBlinkInterval_woChinRest[subjInd][scoreInd])
            
            ipa_subj_easy.append(ipa_woChinRest[subjInd][scoreInd])
            pupilMeanAbsolute_subj_easy.append(pupilMeanAbsolute_woChinRest[subjInd][scoreInd])
            pupilMeanRelative_subj_easy.append(pupilMeanRelative_woChinRest[subjInd][scoreInd])
        
        if scoreInd == len(subjScores)-1:
            scoreComplexity_woChinRest_difficult.append(np.mean(scoreComplexity_subj_diffiult))
            scoreDifficulty_woChinRest_difficult.append(np.mean(scoreDifficulty_subj_difficult))
            scoreAfinn_woChinRest_difficult.append(np.mean(scoreAfinn_subj_difficult))
            
            blinkFrequency_woChinRest_difficult.append(np.mean(blinkFrequency_subj_difficult))
            blinkDurationAverage_woChinRest_difficult.append(np.mean(blinkDurationAverage_subj_difficult))
            interBlinkInterval_woChinRest_difficult.append(np.mean(interBlinkInterval_subj_difficult))
            
            ipa_woChinRest_difficult.append(np.mean(ipa_subj_difficult))
            pupilMeanAbsolute_woChinRest_difficult.append(np.mean(pupilMeanAbsolute_subj_difficult))
            pupilMeanRelative_woChinRest_difficult.append(np.mean(pupilMeanRelative_subj_difficult))
            
            scoreComplexity_woChinRest_easy.append(np.mean(scoreComplexity_subj_easy))
            scoreDifficulty_woChinRest_easy.append(np.mean(scoreDifficulty_subj_easy))
            scoreAfinn_woChinRest_easy.append(np.mean(scoreAfinn_subj_easy))
            
            blinkFrequency_woChinRest_easy.append(np.mean(blinkFrequency_subj_easy))
            blinkDurationAverage_woChinRest_easy.append(np.mean(blinkDurationAverage_subj_easy))
            interBlinkInterval_woChinRest_easy.append(np.mean(interBlinkInterval_subj_easy))
            
            ipa_woChinRest_easy.append(np.mean(ipa_subj_easy))
            pupilMeanAbsolute_woChinRest_easy.append(np.mean(pupilMeanAbsolute_subj_easy))
            pupilMeanRelative_woChinRest_easy.append(np.mean(pupilMeanRelative_subj_easy))
            
            

In [390]:
print('scores')
print(stats.ttest_rel(scoreComplexity_woChinRest_difficult, scoreComplexity_woChinRest_easy))
scoreComplexity_woChinRestDiff = [scoreComplexity_woChinRest_difficult[i] - scoreComplexity_woChinRest_easy[i] for i in range(0, len(scoreComplexity_woChinRest_easy))]
print(np.mean(scoreComplexity_woChinRestDiff), np.std(scoreComplexity_woChinRestDiff))

print(stats.ttest_rel(scoreDifficulty_woChinRest_difficult, scoreDifficulty_woChinRest_easy))
scoreDifficulty_woChinRestDiff = [scoreDifficulty_woChinRest_difficult[i]-scoreDifficulty_woChinRest_easy[i] for i in range(0, len(scoreDifficulty_woChinRest_easy))]
print(np.mean(scoreDifficulty_woChinRestDiff), np.std(scoreDifficulty_woChinRestDiff))

print('blinks')
print(stats.ttest_rel(blinkFrequency_woChinRest_difficult, blinkFrequency_woChinRest_easy))
blinkFrequency_woChinRestDiff = [blinkFrequency_woChinRest_difficult[i]-blinkFrequency_woChinRest_easy[i] for i in range(0, len(blinkFrequency_woChinRest_easy))]
print(np.mean(blinkFrequency_woChinRestDiff), np.mean(blinkFrequency_woChinRestDiff))
print(np.mean(blinkFrequency_woChinRest_difficult), np.std(blinkFrequency_woChinRest_difficult), np.mean(blinkFrequency_woChinRest_easy), np.std(blinkFrequency_woChinRest_difficult))
#print(stats.ttest_rel(blinkDurationAverage_woChinRest_difficult, blinkDurationAverage_woChinRest_easy))
#print(stats.ttest_rel(interBlinkInterval_woChinRest_difficult, interBlinkInterval_woChinRest_easy))

print('pupils')
print(stats.ttest_rel(ipa_woChinRest_difficult, ipa_woChinRest_easy))
ipa_woChinRestDiff = [ipa_woChinRest_difficult[i] - ipa_woChinRest_easy[i] for i in range(0, len(ipa_woChinRest_easy))]
print(np.mean(ipa_woChinRestDiff), np.mean(ipa_woChinRestDiff))

print(stats.ttest_rel(pupilMeanAbsolute_woChinRest_difficult, pupilMeanAbsolute_woChinRest_easy))
pupilMeanAbsolute_woChinRestDiff = [pupilMeanAbsolute_woChinRest_difficult[i]-pupilMeanAbsolute_woChinRest_easy[i] for i in range(0, len(pupilMeanAbsolute_woChinRest_easy))]
print(np.mean(pupilMeanAbsolute_woChinRestDiff), np.std(pupilMeanAbsolute_woChinRestDiff))
print(np.mean(pupilMeanAbsolute_woChinRest_difficult), np.std(pupilMeanAbsolute_woChinRest_difficult), np.mean(pupilMeanAbsolute_woChinRest_easy), np.std(pupilMeanAbsolute_woChinRest_easy))

print(stats.ttest_rel(pupilMeanRelative_woChinRest_difficult, pupilMeanRelative_woChinRest_easy))
pupilMeanRelative_woChinRestDiff = [pupilMeanRelative_woChinRest_difficult[i]-pupilMeanRelative_woChinRest_easy[i] for i in range(0, len(pupilMeanRelative_woChinRest_easy))]
print(np.mean(pupilMeanRelative_woChinRestDiff), np.mean(pupilMeanRelative_woChinRestDiff))


scores
Ttest_relResult(statistic=10.06230589874906, pvalue=0.0005486474801839636)
3.0 0.5962847939999436
Ttest_relResult(statistic=9.130087106158284, pvalue=0.0007985330691842276)
3.1333333333333337 0.6863753427324667
blinks
Ttest_relResult(statistic=2.8772898324005354, pvalue=0.045136030336156444)
0.03743369460466418 0.03743369460466418
0.15369672629454328 0.07763022505357024 0.11626303168987909 0.07763022505357024
pupils
Ttest_relResult(statistic=-3.313182009310074, pvalue=0.029566315355539524)
-0.03527992691324992 -0.03527992691324992
Ttest_relResult(statistic=6.4085506789438265, pvalue=0.003045853466641179)
0.13047466039817426 0.040718929110404534
5.155802430328728 0.7171155644692722 5.025327769930554 0.7332395863400469
Ttest_relResult(statistic=-0.49555269783402534, pvalue=0.6462000977564855)
-0.002421037329469389 -0.002421037329469389


In [88]:
scoreComplexity_difficult = scoreComplexity_woChinRest_difficult + scoreComplexity_wChinRest_difficult
scoreDifficulty_difficult = scoreDifficulty_woChinRest_difficult + scoreDifficulty_wChinRest_difficult
scoreAfinn_difficult = scoreAfinn_woChinRest_difficult + scoreAfinn_wChinRest_difficult
blinkFrequency_difficult = blinkFrequency_woChinRest_difficult + blinkFrequency_wChinRest_difficult
blinkDurationAverage_difficult = blinkDurationAverage_woChinRest_difficult + blinkDurationAverage_wChinRest_difficult
interBlinkInterval_difficult = interBlinkInterval_wChinRest_difficult + interBlinkInterval_woChinRest_difficult
ipa_difficult = ipa_woChinRest_difficult + ipa_wChinRest_difficult
pupilMeanAbsolute_difficult = pupilMeanAbsolute_woChinRest_difficult + pupilMeanAbsolute_wChinRest_difficult
pupilMeanRelative_difficult = pupilMeanRelative_woChinRest_difficult + pupilMeanRelative_wChinRest_difficult


In [89]:
scoreComplexity_easy = scoreComplexity_woChinRest_easy + scoreComplexity_wChinRest_easy
scoreDifficulty_easy = scoreDifficulty_woChinRest_easy + scoreDifficulty_wChinRest_easy
scoreAfinn_easy = scoreAfinn_woChinRest_easy + scoreAfinn_wChinRest_easy
blinkFrequency_easy = blinkFrequency_woChinRest_easy + blinkFrequency_wChinRest_easy
blinkDurationAverage_easy = blinkDurationAverage_woChinRest_easy + blinkDurationAverage_wChinRest_easy
interBlinkInterval_easy = interBlinkInterval_wChinRest_easy + interBlinkInterval_woChinRest_easy
ipa_easy = ipa_woChinRest_easy + ipa_wChinRest_easy
pupilMeanAbsolute_easy = pupilMeanAbsolute_woChinRest_easy + pupilMeanAbsolute_wChinRest_easy
pupilMeanRelative_easy = pupilMeanRelative_woChinRest_easy + pupilMeanRelative_wChinRest_easy



In [388]:
print('scores')
print(stats.ttest_rel(scoreComplexity_difficult, scoreComplexity_easy))
scoreComplexityDiff = [scoreComplexity_difficult[i] - scoreComplexity_easy[i] for i in range(0, len(scoreComplexity_easy))]
print(np.mean(scoreComplexityDiff), np.std(scoreComplexityDiff))

print(stats.ttest_rel(scoreDifficulty_difficult, scoreDifficulty_easy))
scoreDifficultyDiff = [scoreDifficulty_difficult[i]-scoreDifficulty_easy[i] for i in range(0, len(scoreDifficulty_easy))]
print(np.mean(scoreDifficultyDiff), np.std(scoreDifficultyDiff))

print('blinks')
print(stats.ttest_rel(blinkFrequency_difficult, blinkFrequency_easy))
blinkFrequencyDiff = [blinkFrequency_difficult[i]-blinkFrequency_easy[i] for i in range(0, len(blinkFrequency_easy))]
print(np.mean(blinkFrequencyDiff), np.std(blinkFrequencyDiff))
print(stats.ttest_rel(blinkDurationAverage_difficult, blinkDurationAverage_easy))
print(stats.ttest_rel(interBlinkInterval_difficult, interBlinkInterval_easy))

print('pupils')
print(stats.ttest_rel(ipa_difficult, ipa_easy))
ipaDiff = [ipa_difficult[i]-ipa_easy[i] for i in range(0, len(ipa_easy))]
print(np.mean(ipaDiff),np.std(ipaDiff))

print(stats.ttest_rel(pupilMeanAbsolute_difficult, pupilMeanAbsolute_easy))
pupilMeanAbsoluteDiff = [pupilMeanAbsolute_difficult[i]-pupilMeanAbsolute_easy[i] for i in range(0, len(pupilMeanAbsolute_easy))]
print(np.mean(pupilMeanAbsoluteDiff), np.std(pupilMeanAbsoluteDiff))
print(np.mean(pupilMeanAbsolute_difficult), np.std(pupilMeanAbsolute_difficult), np.mean(pupilMeanAbsolute_easy), np.std(pupilMeanAbsolute_easy))

print(stats.ttest_rel(pupilMeanRelative_difficult, pupilMeanRelative_easy))
pupilMeanRelativeDiff = [pupilMeanRelative_difficult[i]-pupilMeanRelative_easy[i] for i in range(0, len(pupilMeanRelative_easy))]
print(np.mean(pupilMeanRelativeDiff), np.mean(pupilMeanRelativeDiff))


scores
Ttest_relResult(statistic=4.270528078814427, pvalue=0.002078697342055484)
2.1666666666666665 1.5220600075774497
Ttest_relResult(statistic=5.335322662698679, pvalue=0.00047136765256018085)
2.5166666666666666 1.4150971698084907
blinks
Ttest_relResult(statistic=1.6833457357225243, pvalue=0.12659779923829523)
0.02461310978737394 0.043864624952062246
Ttest_relResult(statistic=1.0725247044177222, pvalue=0.3113994960516582)
Ttest_relResult(statistic=-0.5310135220138882, pvalue=0.6082623640154365)
pupils
Ttest_relResult(statistic=-2.0667311720247867, pvalue=0.06872696101507828)
-0.02278552288636946 0.033074726691299246
Ttest_relResult(statistic=5.013808159181569, pvalue=0.000725181395188713)
0.12252920253947633 0.0733150523410597
5.098125379139587 0.8155680770055199 4.975596176600112 0.8405411768648375
Ttest_relResult(statistic=1.4787256457462712, pvalue=0.17333801720571815)
0.012470118967577016 0.012470118967577016


In [43]:
ages = [32, 26, 27, 23, 35, 28, 27]
print(np.mean(ages), np.std(ages))

28.285714285714285 3.6922422809485957


In [48]:
print(pupilMean_difficult)

[3.7731059181560602, 5.653053207411607, 5.619072143697089, 5.59973218997363, 5.134048692405252, 3.346867224431937, 5.509625360505399, 5.419750062326152, 5.941361267563984, 4.98463772492477]


In [119]:
scoreDifficulty = scoreDifficulty_wChinRest_difficult + scoreDifficulty_woChinRest_difficult + scoreDifficulty_wChinRest_easy + scoreDifficulty_woChinRest_easy

scoreComplexity = scoreComplexity_wChinRest_difficult + scoreComplexity_woChinRest_difficult + scoreComplexity_wChinRest_easy + scoreComplexity_woChinRest_easy

scoreSubjective = [scoreDifficulty[i] + scoreComplexity[i] for i in range(0, len(scoreDifficulty))]

In [341]:
scoreComplexity_easy = scoreComplexity_wChinRest_easy + scoreComplexity_woChinRest_easy
scoreDifficulty_easy = scoreDifficulty_wChinRest_easy + scoreDifficulty_woChinRest_easy


[6.333333333333333, 4.666666666666667, 6.333333333333333, 4.333333333333333, 4.0, 3.3333333333333335, 5.333333333333333, 5.333333333333333, 3.3333333333333335, 2.3333333333333335]


In [335]:
print(len(scoreDifficulty_wChinRest_difficult))

5


In [117]:
scoreLIX = [3,3,3,3,3,3,3,3,3,3,1,1,1,1,1,1,1,1,1,1]

In [121]:
print(stats.pearsonr(scoreDifficulty, scoreLIX))
print(stats.pearsonr(scoreComplexity, scoreLIX))
print(stats.pearsonr(scoreSubjective, scoreLIX))

(0.7684602759890878, 7.561527021110132e-05)
(0.7053975325688253, 0.0005123206342803252)
(0.7760802481004273, 5.7671324191899106e-05)


In [145]:

chinRest = [1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0]
scoreLIX = [1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]

blinkFrequency_anova = blinkFrequency_wChinRest_difficult + blinkFrequency_woChinRest_difficult + blinkFrequency_wChinRest_easy + blinkFrequency_woChinRest_easy
interBlinkInterval_anova = interBlinkInterval_wChinRest_difficult + interBlinkInterval_woChinRest_difficult + interBlinkInterval_wChinRest_easy + interBlinkInterval_woChinRest_easy

pupilMeanAbsolute_anova = pupilMeanAbsolute_wChinRest_difficult + pupilMeanAbsolute_woChinRest_difficult + pupilMeanAbsolute_wChinRest_easy + pupilMeanAbsolute_woChinRest_easy
pupilMeanRelative_anova = pupilMeanRelative_wChinRest_difficult + pupilMeanRelative_woChinRest_difficult + pupilMeanRelative_wChinRest_easy + pupilMeanRelative_woChinRest_easy

ipa_anova = ipa_wChinRest_difficult + ipa_woChinRest_difficult + ipa_wChinRest_easy + ipa_woChinRest_easy





In [336]:
Df_anova = pd.DataFrame({'blinkFrequency': blinkFrequency_anova, 'interBlinkInterval' : interBlinkInterval_anova, 'pupilAbsolute' : pupilMeanAbsolute_anova, 'pupilRelative':pupilMeanRelative_anova, 'ipa':ipa_anova, 'scoreDifficulty': scoreDifficulty, 'scoreComplexity': scoreComplexity })

In [148]:
# 2-way anova 
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols



In [150]:
model_blinkFrequency = ols('blinkFrequency_anova ~ C(chinRest)*C(scoreLIX)', Df_anova).fit()
table = sm.stats.anova_lm(model_blinkFrequency, typ=2)
print(table)

                           sum_sq    df         F    PR(>F)
C(chinRest)              0.007962   1.0  1.678104  0.213556
C(scoreLIX)              0.003029   1.0  0.638382  0.435998
C(chinRest):C(scoreLIX)  0.000822   1.0  0.173206  0.682807
Residual                 0.075918  16.0       NaN       NaN


In [151]:
model_interBlinkInterval = ols('interBlinkInterval_anova ~ C(chinRest)*C(scoreLIX)', Df_anova).fit()
table = sm.stats.anova_lm(model_interBlinkInterval, typ=2)
print(table)

                             sum_sq    df         F    PR(>F)
C(chinRest)               99.241592   1.0  2.136927  0.163152
C(scoreLIX)               14.620442   1.0  0.314816  0.582513
C(chinRest):C(scoreLIX)    9.736720   1.0  0.209657  0.653194
Residual                 743.060088  16.0       NaN       NaN


In [157]:
model_pupilAbsolute = ols('pupilMeanAbsolute_anova ~ C(chinRest)*C(scoreLIX)', Df_anova).fit()
table = sm.stats.anova_lm(model_pupilAbsolute, typ=2)
print(table)

                            sum_sq    df         F    PR(>F)
C(chinRest)               0.057683   1.0  0.067571  0.798222
C(scoreLIX)               0.075067   1.0  0.087935  0.770633
C(chinRest):C(scoreLIX)   0.000316   1.0  0.000370  0.984896
Residual                 13.658609  16.0       NaN       NaN


In [154]:
model_pupilRelative = ols('pupilMeanRelative_anova ~ C(chinRest)*C(scoreLIX)', Df_anova).fit()
table = sm.stats.anova_lm(model_pupilRelative, typ=2)
print(table)

                           sum_sq    df         F    PR(>F)
C(chinRest)              0.004040   1.0  1.003534  0.331367
C(scoreLIX)              0.000778   1.0  0.193155  0.666182
C(chinRest):C(scoreLIX)  0.001109   1.0  0.275437  0.606904
Residual                 0.064406  16.0       NaN       NaN


In [155]:
model_ipa = ols('ipa_anova ~ C(chinRest)*C(scoreLIX)', Df_anova).fit()
table = sm.stats.anova_lm(model_ipa, typ=2)
print(table)

                           sum_sq    df         F    PR(>F)
C(chinRest)              0.000551   1.0  0.788740  0.387638
C(scoreLIX)              0.002596   1.0  3.712631  0.071953
C(chinRest):C(scoreLIX)  0.000781   1.0  1.116336  0.306402
Residual                 0.011187  16.0       NaN       NaN


In [337]:
model_Complexity= ols('scoreComplexity ~ C(chinRest)*C(scoreLIX)', Df_anova).fit()
table = sm.stats.anova_lm(model_Complexity, typ=2)
print(table)

                            sum_sq    df          F    PR(>F)
C(chinRest)               0.005556   1.0   0.004396  0.947961
C(scoreLIX)              23.472222   1.0  18.571429  0.000540
C(chinRest):C(scoreLIX)   3.472222   1.0   2.747253  0.116896
Residual                 20.222222  16.0        NaN       NaN


In [338]:
model_Difficulty= ols('scoreDifficulty ~ C(chinRest)*C(scoreLIX)', Df_anova).fit()
table = sm.stats.anova_lm(model_Difficulty, typ=2)
print(table)

                            sum_sq    df          F    PR(>F)
C(chinRest)               1.701389   1.0   1.483051  0.240953
C(scoreLIX)              31.668056   1.0  27.604116  0.000079
C(chinRest):C(scoreLIX)   1.901389   1.0   1.657385  0.216269
Residual                 18.355556  16.0        NaN       NaN


In [380]:
# create data for plotting
blinkFrequency_plotMean = [[np.mean(blinkFrequency_easy), np.mean(blinkFrequency_wChinRest_easy), np.mean(blinkFrequency_woChinRest_easy)],
                          [ np.mean(blinkFrequency_difficult), np.mean(blinkFrequency_wChinRest_difficult), np.mean(blinkFrequency_woChinRest_difficult)]]

blinkFrequency_plotStd = [[np.std(blinkFrequency_easy), np.std(blinkFrequency_wChinRest_easy), np.std(blinkFrequency_woChinRest_easy)],
                          [ np.std(blinkFrequency_difficult), np.std(blinkFrequency_wChinRest_difficult), np.std(blinkFrequency_woChinRest_difficult)]]

# plots
fig = plt.figure()
ax = fig.add_subplot(111)
plt.rcParams.update({'font.size': 15})
X = np.arange(3)
p1 = ax.bar(X + 0.00, blinkFrequency_plotMean[0], yerr = blinkFrequency_plotStd[0], color = 'orange', width = 0.20)
p2 = ax.bar(X + 0.25, blinkFrequency_plotMean[1], yerr = blinkFrequency_plotStd[1], color = 'g', width = 0.20)

ax.set_ylabel('Blink frequency [in Hz]')
ax.set_xticks([0.125, 1.125, 2.125])
#ax.set_xticklabels({'overall', 'with chin rest', 'without chin rest'})
plt.xticks([0.125, 1.125, 2.125], ['overall', 'with chin rest', 'without chin rest'])

blinkFrequency_stdmax = max(max(blinkFrequency_plotStd))
blinkFrequency_meanmax = max(max(blinkFrequency_plotMean))
ax.set_ylim([0, blinkFrequency_meanmax + blinkFrequency_stdmax+0.11])

ax.legend((p1[0], p2[0]), ('easy', 'difficult') )

ax.text((2.125), np.mean(blinkFrequency_woChinRest_easy) + np.std(blinkFrequency_woChinRest_easy)+0.04, "*", ha='center', va='bottom', color='k', fontsize = 20)
plt.tight_layout()
fig.savefig(r'C:\DTU\Results\201812_PilotExptForMovement\Paper\CHI2019\blinkFrequency.png', dpi = 300, bbox_to_anchor = (0.9, 0.1))

#plt.savefig(r'C:\DTU\Results\201812_PilotExptForMovement\Paper\CHI2019\blinkFrequency.png', format='png', dpi=1000)


In [284]:
a = ax.get_xticks()
print(a)

[0.125 1.125 2.125]


In [259]:
# create data for plotting
interBlinkInterval_plotMean = [[np.mean(interBlinkInterval_easy), np.mean(interBlinkInterval_wChinRest_easy), np.mean(interBlinkInterval_woChinRest_easy)],
                          [ np.mean(interBlinkInterval_difficult), np.mean(interBlinkInterval_wChinRest_difficult), np.mean(interBlinkInterval_woChinRest_difficult)]]

interBlinkInterval_plotStd = [[np.std(interBlinkInterval_easy), np.std(interBlinkInterval_wChinRest_easy), np.std(interBlinkInterval_woChinRest_easy)],
                          [ np.std(interBlinkInterval_difficult), np.std(interBlinkInterval_wChinRest_difficult), np.std(interBlinkInterval_woChinRest_difficult)]]

# plots
fig = plt.figure()
ax = fig.add_subplot(111)
X = np.arange(3)
ax.bar(X + 0.00, interBlinkInterval_plotMean[0], yerr = interBlinkInterval_plotStd[0], color = 'orange', width = 0.20)
ax.bar(X + 0.25, interBlinkInterval_plotMean[1], yerr = interBlinkInterval_plotStd[1], color = 'g', width = 0.20)

ax.set_ylabel('Inter-blink interval [in s]')
#ax.set_xticks([0.125, 1.125, 2.125], ['overall', 'with chin rest', 'without chin rest'])
plt.xticks([0.125, 1.125, 2.125], ['overall', 'with chin rest', 'without chin rest'])

interBlinkInterval_max = max(max(interBlinkInterval_plotStd))
ax.set_ylim([-1, interBlinkInterval_max+15])

#ax.set_.text((2.125), np.mean(interBlinkInterval_woChinRest_easy) + np.std(interBlinkInterval_woChinRest_easy)+0.03, "*", ha='center', va='bottom', color='k', fontsize = 20)



(-1, 23.58777607646514)

In [381]:
# create data for plotting
pupilMeanAbsolute_plotMean = [[np.mean(pupilMeanAbsolute_easy), np.mean(pupilMeanAbsolute_wChinRest_easy), np.mean(pupilMeanAbsolute_woChinRest_easy)],
                          [ np.mean(pupilMeanAbsolute_difficult), np.mean(pupilMeanAbsolute_wChinRest_difficult), np.mean(pupilMeanAbsolute_woChinRest_difficult)]]

pupilMeanAbsolute_plotStd = [[np.std(pupilMeanAbsolute_easy), np.std(pupilMeanAbsolute_wChinRest_easy), np.std(pupilMeanAbsolute_woChinRest_easy)],
                          [ np.std(pupilMeanAbsolute_difficult), np.std(pupilMeanAbsolute_wChinRest_difficult), np.std(pupilMeanAbsolute_woChinRest_difficult)]]

# plots
fig = plt.figure()
ax = fig.add_subplot(111)
plt.rcParams.update({'font.size': 15})
X = np.arange(3)
p1 = ax.bar(X + 0.00, pupilMeanAbsolute_plotMean[0], yerr = pupilMeanAbsolute_plotStd[0], color = 'orange', width = 0.20)
p2 = ax.bar(X + 0.25, pupilMeanAbsolute_plotMean[1], yerr = pupilMeanAbsolute_plotStd[1], color = 'g', width = 0.20)


ax.set_ylabel('Absolute pupil size [in mm]')
ax.set_xticks([0.125, 1.125, 2.125])
#ax.set_xticklabels({'overall', 'with chin rest', 'without chin rest'})
plt.xticks([0.125, 1.125, 2.125], ['overall', 'with chin rest', 'without chin rest'])

pupilMeanAbsolute_stdmax = max(max(pupilMeanAbsolute_plotStd))
pupilMeanAbsolute_meanmax = max(max(pupilMeanAbsolute_plotMean))
ax.set_ylim([0, pupilMeanAbsolute_meanmax+pupilMeanAbsolute_stdmax+3])

ax.legend((p1[0], p2[0]), ('easy', 'difficult') )

ax.text((0.125), np.mean(pupilMeanAbsolute_easy) + np.std(pupilMeanAbsolute_easy)+0.5, "***", ha='center', va='bottom', color='k', fontsize = 20)
ax.text((2.125), np.mean(pupilMeanAbsolute_woChinRest_easy) + np.std(pupilMeanAbsolute_woChinRest_easy)+0.5, "**", ha='center', va='bottom', color='k', fontsize = 20)

plt.tight_layout()
fig.savefig(r'C:\DTU\Results\201812_PilotExptForMovement\Paper\CHI2019\pupilSize.png', dpi = 300, bbox_to_anchor = (0.95, 0.1))
#plt.savefig(r'C:\DTU\Results\201812_PilotExptForMovement\Paper\CHI2019\pupilSize', format='eps', dpi=1000)

In [360]:
# create data for plotting
ipa_plotMean = [[np.mean(ipa_easy), np.mean(ipa_wChinRest_easy), np.mean(ipa_woChinRest_easy)],
                          [ np.mean(ipa_difficult), np.mean(ipa_wChinRest_difficult), np.mean(ipa_woChinRest_difficult)]]

ipa_plotStd = [[np.std(ipa_wChinRest_easy), np.std(ipa_wChinRest_easy), np.std(ipa_woChinRest_easy)],
                          [ np.std(ipa_difficult), np.std(ipa_wChinRest_difficult), np.std(ipa_woChinRest_difficult)]]

# plots
fig = plt.figure()
ax = fig.add_subplot(111)
X = np.arange(3)
p1 = ax.bar(X + 0.00, ipa_plotMean[0], yerr = ipa_plotStd[0], color = 'orange', width = 0.20)
p2 = ax.bar(X + 0.25, ipa_plotMean[1], yerr = ipa_plotStd[1], color = 'g', width = 0.20)

ax.set_ylabel('IPA values [in Hz]')
ax.set_xticks([0.125, 1.125, 2.125])
#ax.set_xticklabels({'overall', 'with chin rest', 'without chin rest'})
plt.xticks([0.125, 1.125, 2.125], ['overall', 'with chin rest', 'without chin rest'])

ipa_stdmax = max(max(ipa_plotStd))
ipa_meanmax = max(max(ipa_plotMean))
ax.set_ylim([0, ipa_meanmax+ipa_stdmax+0.025])

ax.legend((p1[0], p2[0]), ('easy', 'difficult') )
plt.tight_layout()
ax.text((2.125), np.mean(ipa_woChinRest_easy) + np.std(ipa_woChinRest_easy)+0.01, "*", ha='center', va='bottom', color='k', fontsize = 20)
plt.savefig(r'C:\DTU\Results\201812_PilotExptForMovement\Paper\CHI2019\ipa.png', format='png', dpi=1000)


In [385]:
# create data for plotting
scoreComplexity_plotMean = [[np.mean(scoreComplexity_easy), np.mean(scoreComplexity_wChinRest_easy), np.mean(scoreComplexity_woChinRest_easy)],
                          [ np.mean(scoreComplexity_difficult), np.mean(scoreComplexity_wChinRest_difficult), np.mean(scoreComplexity_woChinRest_difficult)]]

scoreComplexity_plotStd = [[np.std(scoreComplexity_easy), np.std(scoreComplexity_wChinRest_easy), np.std(scoreComplexity_woChinRest_easy)],
                          [ np.std(scoreComplexity_difficult), np.std(scoreComplexity_wChinRest_difficult), np.std(scoreComplexity_woChinRest_difficult)]]

# plots
fig = plt.figure()
ax = fig.add_subplot(111)
X = np.arange(3)
p1 = ax.bar(X + 0.00, scoreComplexity_plotMean[0], yerr = scoreComplexity_plotStd[0], color = 'orange', width = 0.20)
p2 = ax.bar(X + 0.25, scoreComplexity_plotMean[1], yerr = scoreComplexity_plotStd[1], color = 'g', width = 0.20)

ax.set_ylabel('Expected task complexity score [A.U.]')
ax.set_xticks([0.125, 1.125, 2.125])
#ax.set_xticklabels({'overall', 'with chin rest', 'without chin rest'})
plt.xticks([0.125, 1.125, 2.125], ['overall', 'with chin rest', 'without chin rest'])

scoreComplexity_stdmax = max(max(scoreComplexity_plotStd))
scoreComplexity_meanmax = max(max(scoreComplexity_plotMean))
ax.set_ylim([0, scoreComplexity_meanmax+scoreComplexity_stdmax+4])

ax.legend((p1[0], p2[0]), ('easy', 'difficult') )
plt.tight_layout()
ax.text((0.125), np.mean(scoreComplexity_easy) + np.std(scoreComplexity_easy)+3, "**", ha='center', va='bottom', color='k', fontsize = 20)
ax.text((2.125), np.mean(scoreComplexity_easy) + np.std(scoreComplexity_easy)+3, "***", ha='center', va='bottom', color='k', fontsize = 20)

fig.savefig(r'C:\DTU\Results\201812_PilotExptForMovement\Paper\CHI2019\Complexity.png', dpi = 300, bbox_to_anchor = (0.95, 0.1))


In [387]:
# create data for plotting
scoreDifficulty_plotMean = [[np.mean(scoreDifficulty_easy), np.mean(scoreDifficulty_wChinRest_easy), np.mean(scoreDifficulty_woChinRest_easy)],
                          [ np.mean(scoreDifficulty_difficult), np.mean(scoreDifficulty_wChinRest_difficult), np.mean(scoreDifficulty_woChinRest_difficult)]]

scoreDifficulty_plotStd = [[np.std(scoreDifficulty_easy), np.std(scoreDifficulty_wChinRest_easy), np.std(scoreDifficulty_woChinRest_easy)],
                          [ np.std(scoreDifficulty_difficult), np.std(scoreDifficulty_wChinRest_difficult), np.std(scoreDifficulty_woChinRest_difficult)]]

# plots
fig = plt.figure()
ax = fig.add_subplot(111)
X = np.arange(3)
p1 = ax.bar(X + 0.00, scoreDifficulty_plotMean[0], yerr = scoreDifficulty_plotStd[0], color = 'orange', width = 0.20)
p2 = ax.bar(X + 0.25, scoreDifficulty_plotMean[1], yerr = scoreDifficulty_plotStd[1], color = 'g', width = 0.20)

ax.set_ylabel('Subjective task difficulty score [A.U.]')
ax.set_xticks([0.125, 1.125, 2.125])
#ax.set_xticklabels({'overall', 'with chin rest', 'without chin rest'})
plt.xticks([0.125, 1.125, 2.125], ['overall', 'with chin rest', 'without chin rest'])

scoreDifficulty_stdmax = max(max(scoreDifficulty_plotStd))
scoreDifficulty_meanmax = max(max(scoreDifficulty_plotMean))
ax.set_ylim([0, scoreDifficulty_meanmax+scoreDifficulty_stdmax+4])

ax.legend((p1[0], p2[0]), ('easy', 'difficult') )
plt.tight_layout()
ax.text((0.125), np.mean(scoreDifficulty_woChinRest_easy) + np.std(scoreDifficulty_woChinRest_easy)+3.7, "***", ha='center', va='bottom', color='k', fontsize = 20)
ax.text((2.125), np.mean(scoreDifficulty_woChinRest_easy) + np.std(scoreDifficulty_woChinRest_easy)+3.7, "***", ha='center', va='bottom', color='k', fontsize = 20)

fig.savefig(r'C:\DTU\Results\201812_PilotExptForMovement\Paper\CHI2019\Difficulty.png', dpi = 300, bbox_to_anchor = (0.95, 0.1))


In [170]:
[np.mean(blinkFrequency_easy), np.mean(blinkFrequency_difficult)]

[0.1027204438565014, 0.12733355364387536]